In [1]:
import torch
import json
from generate_transforms import (
    generate_detection_train_transform,
    generate_detection_train_transform_using_coord,
)
from monai.data import Dataset, load_decathlon_datalist
from monai.transforms import ScaleIntensityRanged
from monai.utils import set_determinism

set_determinism(seed=24)

environment_file = "/workspace/Code/tutorials/detection/config/environment_luna16_fold0.json"
config_file = "/workspace/Code/tutorials/detection/config/config_train_luna16_16g.json"
env_dict = json.load(open(environment_file, "r"))
config_dict = json.load(open(config_file, "r"))

args = {}
for k, v in env_dict.items():
    args[k] = v
for k, v in config_dict.items():
    args[k] = v

intensity_transform = ScaleIntensityRanged(
    keys=["image"],
    a_min=-1024,
    a_max=300.0,
    b_min=0.0,
    b_max=1.0,
    clip=True,
)

train_transforms_old = generate_detection_train_transform(
    "image",
    "box",
    "label",
    args["gt_box_mode"],
    intensity_transform,
    args["patch_size"],
    args["batch_size"],
    affine_lps_to_ras=True,
    amp=True,
)

train_transforms_new = generate_detection_train_transform_using_coord(
    "image",
    "box",
    "label",
    "point",
    args["gt_box_mode"],
    intensity_transform,
    args["patch_size"],
    args["batch_size"],
    affine_lps_to_ras=True,
    amp=True,
)

train_data = load_decathlon_datalist(
    args["data_list_file_path"],
    is_segmentation=True,
    data_list_key="training",
    base_dir=args["data_base_dir"],
)
train_ds_old = Dataset(
    data=train_data,
    transform=train_transforms_old,
)
train_ds_new = Dataset(
    data=train_data,
    transform=train_transforms_new,
)

/usr/local/lib/python3.10/dist-packages/ignite/handlers/checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for j, i in enumerate(train_ds_old):
    print(f"old version {j}")
    print(i[0]["box"])

for j, i in enumerate(train_ds_new):
    print(f"new version {j}")
    print(i[0]["box"])

old version 0
metatensor([[117.,   0.,  63., 122.,   1.,  65.],
        [130.,  27.,  42., 144.,  41.,  51.]], dtype=torch.float16)
old version 1
metatensor([], size=(0, 6), dtype=torch.float16)
new version 0
metatensor([[117.,   0.,  63., 122.,   1.,  65.],
        [130.,  27.,  42., 144.,  41.,  51.]])
new version 1
metatensor([], size=(0, 6))
